# Example 2 Darcy interface flow evaluation

In [1]:
import sys
sys.path.append("../")
from libs import *
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Package not found.
Please install Plotly for showing mesh and solutions.


# Fourier

In [2]:
subsample_nodes = 3
subsample_attn = 6
train_path = os.path.join(DATA_PATH, 'piececonst_r421_N1024_smooth1.mat')
test_path = os.path.join(DATA_PATH, 'piececonst_r421_N1024_smooth2.mat')
train_dataset = DarcyDataset(data_path=train_path,
                                subsample_attn=subsample_attn,
                                subsample_nodes=subsample_nodes,
                                train_data=True,
                                train_len=1024,)

valid_dataset = DarcyDataset(data_path=test_path,
                                normalizer_x=train_dataset.normalizer_x,
                                subsample_attn=subsample_attn,
                                subsample_nodes=subsample_nodes,
                                train_data=False,
                                valid_len=100)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False,
                            drop_last=False, pin_memory=True)

Loading piececonst_r421_N1024_smooth1.mat: start at 1701396164.21;
LOCAL RAM USAGE AT START: 0.34 GB
Loading piececonst_r421_N1024_smooth1.mat: done at 1701396180.98 (16.761958 secs elapsed);
LOCAL RAM USAGE AT END: 1.88GB (+1.54GB)


Loading piececonst_r421_N1024_smooth2.mat: start at 1701396182.34;
LOCAL RAM USAGE AT START: 2.34 GB
Loading piececonst_r421_N1024_smooth2.mat: done at 1701396199.29 (16.946509 secs elapsed);
LOCAL RAM USAGE AT END: 3.86GB (+1.52GB)




In [3]:
n_grid = int(((421 - 1)/subsample_nodes) + 1)
n_grid_c = int(((421 - 1)/subsample_attn) + 1)
h = 1/n_grid
metric_func = WeightedL2Loss2d(regularizer=False, h=h)
model = torch.load(os.path.join(MODEL_PATH, 'darcy_141_6ft_256d_qkv_32f_2023-11-20.pt'))
model.to(device)
model.eval()
val_metric = validate_epoch_darcy(model, metric_func, valid_loader, device)
print(f"\nModel's validation metric: {val_metric}")


Model's validation metric: {'metric': 0.008343311911448836}


# Galerkin

In [4]:
# subsample_nodes = 3
# subsample_attn = 6
# train_path = os.path.join(DATA_PATH, 'piececonst_r421_N1024_smooth1.mat')
# test_path = os.path.join(DATA_PATH, 'piececonst_r421_N1024_smooth2.mat')
# train_dataset = DarcyDataset(data_path=train_path,
#                                 subsample_attn=subsample_attn,
#                                 subsample_nodes=subsample_nodes,
#                                 train_data=True,
#                                 train_len=1024,)

# valid_dataset = DarcyDataset(data_path=test_path,
#                                 normalizer_x=train_dataset.normalizer_x,
#                                 subsample_attn=subsample_attn,
#                                 subsample_nodes=subsample_nodes,
#                                 train_data=False,
#                                 valid_len=100)
# valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False,
#                             drop_last=False, pin_memory=True)

In [5]:
n_grid = int(((421 - 1)/subsample_nodes) + 1)
n_grid_c = int(((421 - 1)/subsample_attn) + 1)
h = 1/n_grid
metric_func = WeightedL2Loss2d(regularizer=False, h=h)
model = torch.load(os.path.join(MODEL_PATH, 'darcy_141_6gt_256d_qkv_32f_2023-11-20.pt'))
model.to(device)
model.eval()
val_metric = validate_epoch_darcy(model, metric_func, valid_loader, device)
print(f"\nModel's validation metric: {val_metric}")


Model's validation metric: {'metric': 0.008848846592009067}


# Lrk

In [6]:
# subsample_nodes = 3
# subsample_attn = 6
# train_path = os.path.join(DATA_PATH, 'piececonst_r421_N1024_smooth1.mat')
# test_path = os.path.join(DATA_PATH, 'piececonst_r421_N1024_smooth2.mat')
# train_dataset = DarcyDataset(data_path=train_path,
#                                 subsample_attn=subsample_attn,
#                                 subsample_nodes=subsample_nodes,
#                                 train_data=True,
#                                 train_len=1024,)

# valid_dataset = DarcyDataset(data_path=test_path,
#                                 normalizer_x=train_dataset.normalizer_x,
#                                 subsample_attn=subsample_attn,
#                                 subsample_nodes=subsample_nodes,
#                                 train_data=False,
#                                 valid_len=100)
# valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False,
#                             drop_last=False, pin_memory=True)

# n_grid = int(((421 - 1)/subsample_nodes) + 1)
# n_grid_c = int(((421 - 1)/subsample_attn) + 1)
# downsample, upsample = DarcyDataset.get_scaler_sizes(n_grid, n_grid_c)


In [7]:
n_grid = int(((421 - 1)/subsample_nodes) + 1)
n_grid_c = int(((421 - 1)/subsample_attn) + 1)
h = 1/n_grid
metric_func = WeightedL2Loss2d(regularizer=False, h=h)
model = torch.load(os.path.join(MODEL_PATH, 'darcy_141_6Lt_256d_qkv_32f_2023-11-20.pt'))
model.to(device)
print(len(model.v))
model.eval()
val_metric = validate_epoch_darcy(model, metric_func, valid_loader, device)
print(f"\nModel's validation metric: {val_metric}")

22

Model's validation metric: {'metric': 0.008303205585107207}


In [8]:
# sample = next(iter(valid_loader))
# a = sample['node']
# pos = sample['pos']
# u = sample['target']
# grid = sample['grid']
# with torch.no_grad():
#     model.eval()
#     out_ = model(a.to(device), None, pos.to(device), grid.to(device))
#     preds = out_['preds']

In [9]:
# i = 0
# z = preds[i, ..., 0].cpu().numpy()
# z_true = u[i, ..., 0].cpu().numpy()
# _ = showcontour(z, width=300, height=300,)
# _ = showcontour(z_true, width=300, height=300,)
# print("Relative error: ", np.linalg.norm(z-z_true)/np.linalg.norm(z_true), '\n\n')

In [10]:
# i = 1
# z = preds[i, ..., 0].cpu().numpy()
# z_true = u[i, ..., 0].cpu().numpy()
# _ = showcontour(z, width=300, height=300,)
# _ = showcontour(z_true, width=300, height=300,)
# print("Relative error: ", np.linalg.norm(z-z_true)/np.linalg.norm(z_true), '\n\n')

In [11]:
# i = 2
# z = preds[i, ..., 0].cpu().numpy()
# z_true = u[i, ..., 0].cpu().numpy()
# _ = showcontour(z, width=300, height=300,)
# _ = showcontour(z_true, width=300, height=300,)
# print("Relative error: ", np.linalg.norm(z-z_true)/np.linalg.norm(z_true), '\n\n')

In [12]:
# i = 3
# z = preds[i, ..., 0].cpu().numpy()
# z_true = u[i, ..., 0].cpu().numpy()
# _ = showcontour(z, width=300, height=300,)
# _ = showcontour(z_true, width=300, height=300,)
# print("Relative error: ", np.linalg.norm(z-z_true)/np.linalg.norm(z_true), '\n\n')